### This notebook will be mainly used for the Coursera Capstone project

In [3]:
import pandas as pd
import numpy as np

In [3]:
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [437]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 999

In [ ]:
#! pip install lxml

In [4]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [7]:
df.head(5), df.shape

(  Postal Code           Borough               Neighborhood
 0         M1A      Not assigned                        NaN
 1         M2A      Not assigned                        NaN
 2         M3A        North York                  Parkwoods
 3         M4A        North York           Victoria Village
 4         M5A  Downtown Toronto  Regent Park, Harbourfront,
 (180, 3))

### Filter dataframe to exclude Borough Not assigned. Removed 77 rows

In [8]:
df = df[df['Borough'] != 'Not assigned']
df.shape

(103, 3)

## Since working instructions said that possible duplicates in postal index can happen, check for uniqueness

In [9]:
df['Postal Code'].is_unique ## no dublicates, no need to combine data from multiple rows

True

###  Check if "Not assigned" or its modifications is in Neighborhood

In [25]:
df['Neighborhood'].isin(["not assigned","Not assigned","Not Assigned"]).sum() # everything is assigned

0

# Final dataframe

In [26]:
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [27]:
df.shape #final shape

(103, 3)

### Part 2.Postal codes into Geospatial data

In [29]:
!pip install geocoder

     |████████████████████████████████| 102kB 16.1MB/s ta 0:00:01
     |████████████████████████████████| 92kB 6.7MB/s  eta 0:00:01
     |████████████████████████████████| 829kB 7.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [30]:
import geocoder

In [56]:
### get all data from arcgis into dict

geo_dict = {}
for postal_code in df['Postal Code']:
    g = geocoder.arcgis('{},Toronto,Ontario'.format(postal_code))
    if g.ok:
        geo_dict[postal_code] = g.json
    else:
        geo_dict[postal_code] = 'error'
   

In [158]:
geo_dict.keys(), geo_dict['M3A']  # example of dict and its keys

(dict_keys(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B', 'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C', 'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H', 'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J', 'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L', 'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M', 'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N', 'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R', 'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S', 'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V', 'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X', 'M4Y', 'M7Y', 'M8Y', 'M8Z']),
 {'address': 'M3A',
  'bbox': {'northeast': [43.77693455500008, -79.31164142299997],
   'southwest': [43.72893455500008, -79.35964142299997]},
  'confidence': 6,
  'lat': 43.75293455500008,
  'lng': -79.33564142299997,
  'ok': True,
  'quality': 'Postal',
  'raw': 

In [120]:
## add coordinates for each row
def parse_lnglat(postal_code, geo_dict):
    return geo_dict[postal_code]['lat'], geo_dict[postal_code]['lng']

In [ ]:
## using apply pandas method create dataframe with lat/long data and concat with original df from part 1

In [166]:
df_mapped = \
    pd.concat(
    [df.reset_index(),
    (pd.DataFrame(df['Postal Code'].apply(
        parse_lnglat, args=(geo_dict,)).values.tolist(), columns=['Latitude','Longitude']))
    ], axis = 1).drop(columns=['index'])


# Part 2.Final dataframe with Latitude/Longitude

In [218]:
df_mapped.to_csv('data_mapped.csv')
df_mapped

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653340,-79.509766
99,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
100,M7Y,East Toronto,Business reply mail Processing Centre,43.648700,-79.385450
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632798,-79.493017


# Neighborhoods visuals

In [175]:
## display map of Neighborhoods
toronto_data = geocoder.arcgis('Toronto,Ontario')

In [196]:
df_mapped.head(3)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041


In [178]:
import folium

In [252]:
toronto_map = folium.Map(
    location=[toronto_data.lat,toronto_data.lng],zoom_start=9)

## add markers:

for row in df_mapped.itertuples():
    # itertupes doesn't like 'Postal Code since it has space and not valid
    # Pythin identifier and renamed that to _1'
    label_txt = (getattr(row,'_1'), getattr(row,'Borough'), getattr(row,'Neighborhood'))
    label = 'Postal: {}; Borough: {}; Neighboorhood: {}'.format(*label_txt)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [getattr(row,'Latitude'), getattr(row,'Longitude')],
        radius = 7,
        popup=label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(toronto_map)
toronto_map

###  Load Foursquare data

In [308]:
import requests
from pandas.io.json import json_normalize

In [315]:
### Load  FourSquare API credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
#LIMIT = 30
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

In [305]:
## get venues from FourSquare
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_mapped.loc[0,'Latitude'], 
    df_mapped.loc[0,'Longitude'], 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=C3WMWMMIUBZNY3SLBDQXTQPTV1R44N1TR3LPEIMAL0MGKEZB&client_secret=UAWABDH1O2PW1TTP10PFEMXO2B41IQRLN0VJRAN55MVU1YQB&v=20180604&ll=43.75293455500008,-79.33564142299997&radius=2000&limit=100&time=any&day=any'

In [306]:
results = requests.get(url).json()

In [307]:
len(results['response']['groups'][0]['items'])

100

In [348]:
def getNearByVenues(names, lats, longs, radius=1000):
    
    venues_list = []
    LIMIT=50
    for name,lat,lng in zip(names,lats,longs):
        print (name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&time=any&day=any'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
        
        print ('Processing:' + str(name))
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #manually parse the json response
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        
    

In [349]:
# Run with Toronto venues:
toronto_venues = getNearByVenues(names = df_mapped['Neighborhood'],
                                lats = df_mapped['Latitude'],
                                longs = df_mapped['Longitude'])


Parkwoods
Processing:Parkwoods
Victoria Village
Processing:Victoria Village
Regent Park, Harbourfront
Processing:Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Processing:Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Processing:Queen's Park, Ontario Provincial Government
Islington Avenue
Processing:Islington Avenue
Malvern, Rouge
Processing:Malvern, Rouge
Don Mills
Processing:Don Mills
Parkview Hill, Woodbine Gardens
Processing:Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Processing:Garden District, Ryerson
Glencairn
Processing:Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Processing:West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Processing:Rouge Hill, Port Union, Highland Creek
Don Mills
Processing:Don Mills
Woodbine Heights
Processing:Woodbine Heights
St. James Town
Processing:St. James Town
Humewood-Cedarvale
Processing:Humewood-Ced

In [420]:
#toronto_venues.to_csv('toronto_venues.csv')
toronto_venues = pd.read_csv('toronto_venues.csv')

In [421]:
toronto_venues

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Parkwoods,43.752935,-79.335641,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
1,1,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
2,2,Parkwoods,43.752935,-79.335641,TD Canada Trust,43.753253,-79.347851,Bank
3,3,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,4,Parkwoods,43.752935,-79.335641,TTC Stop #9075,43.757596,-79.338155,Train Station
...,...,...,...,...,...,...,...,...
3372,3372,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,7-Eleven,43.629107,-79.517431,Convenience Store
3373,3373,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
3374,3374,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,Milestones,43.623197,-79.515430,Restaurant
3375,3375,"Mimico NW, The Queensway West, South of Bloor,...",43.625490,-79.526000,Tim Hortons,43.620900,-79.515340,Coffee Shop


In [422]:
toronto_venues.groupby('Neighborhood').count()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Agincourt,41,41,41,41,41,41,41
"Alderwood, Long Branch",32,32,32,32,32,32,32
"Bathurst Manor, Wilson Heights, Downsview North",30,30,30,30,30,30,30
Bayview Village,13,13,13,13,13,13,13
"Bedford Park, Lawrence Manor East",37,37,37,37,37,37,37
...,...,...,...,...,...,...,...
"Willowdale, Newtonbrook",20,20,20,20,20,20,20
Woburn,7,7,7,7,7,7,7
Woodbine Heights,50,50,50,50,50,50,50


In [423]:
# Total venue types
toronto_venues['Venue Category'].unique()

array(['Golf Course', 'Park', 'Bank', 'Food & Drink Shop',
       'Train Station', 'Road', 'Bus Stop', 'Coffee Shop',
       'Middle Eastern Restaurant', 'Hockey Arena', 'Thai Restaurant',
       'Grocery Store', 'Portuguese Restaurant', 'Sandwich Place',
       'Pizza Place', 'Bus Line', 'Mediterranean Restaurant', 'Spa',
       'Sporting Goods Shop', 'Chinese Restaurant', 'Wings Joint',
       'Chocolate Shop', 'Athletics & Sports', 'Performing Arts Venue',
       'Historic Site', 'French Restaurant', 'Tech Startup', 'Café',
       'Distribution Center', 'Restaurant', 'Pub', 'Theater',
       'Farmers Market', 'Dessert Shop', 'Circus', 'Bakery', 'Shoe Store',
       'BBQ Joint', 'Diner', 'Gym / Fitness Center', 'Ice Cream Shop',
       'Hotel', 'Mexican Restaurant', 'Breakfast Spot', 'Liquor Store',
       'Hotel Bar', 'Asian Restaurant', 'Brewery', 'Cosmetics Shop',
       'Gastropub', 'Clothing Store', 'Shopping Mall',
       'Furniture / Home Store', 'Toy / Game Store', "Men's Sto

In [426]:
# also noticed that foursquare api returned 'neighboorhood' value in venue category above for some reason. need to clean it, 
# because it this value will mess up with the next step OHE 
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood']
toronto_venues.shape

(3367, 8)

In [441]:
toronto_ohe = pd.get_dummies(toronto_venues['Venue Category'], prefix='', prefix_sep='')
toronto_ohe['Neighborhood'] = toronto_venues['Neighborhood']

column_order = [toronto_ohe.columns[-1]] + list(toronto_ohe.columns[:-1])

toronto_ohe = toronto_ohe[column_order]
toronto_ohe.head(5), toronto_ohe.shape

(  Neighborhood  Accessories Store  Afghan Restaurant  Airport  \
 0    Parkwoods                  0                  0        0   
 1    Parkwoods                  0                  0        0   
 2    Parkwoods                  0                  0        0   
 3    Parkwoods                  0                  0        0   
 4    Parkwoods                  0                  0        0   
 
    American Restaurant  Antique Shop  Argentinian Restaurant  Art Gallery  \
 0                    0             0                       0            0   
 1                    0             0                       0            0   
 2                    0             0                       0            0   
 3                    0             0                       0            0   
 4                    0             0                       0            0   
 
    Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
 0           0                    0                 0  

In [447]:
# group by neighborhood and see the frequency:

toronto_grouped = toronto_ohe.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Storage Facility,Supermarket,

In [449]:
toronto_grouped.shape

(97, 314)

In [465]:
# Just checking how sorting works in the sample work


hood = 'Agincourt'
_t = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
_t.columns = ['venue','freq']
_t = _t.loc[1:]
_t['freq'] = _t['freq'].astype(float)
_t = _t.round({'freq' :2})
_t.sort_values('freq', ascending=False).reset_index(drop=True).head(10)

,venue,freq
0,Chinese Restaurant,0.17
1,Shopping Mall,0.10
2,Bakery,0.05
3,Sandwich Place,0.05
4,Supermarket,0.02
5,Sushi Restaurant,0.02
6,Malay Restaurant,0.02
7,Coffee Shop,0.02
8,Lounge,0.02
9,Noodle House,0.02


In [493]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
        
venue_sorted = pd.DataFrame(columns=columns)
venue_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
venue_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Alderwood, Long Branch",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Bathurst Manor, Wilson Heights, Downsview North",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bayview Village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bedford Park, Lawrence Manor East",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Berczy Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Birch Cliff, Cliffside West",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Brockton, Parkdale Village, Exhibition Place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Business reply mail Processing Centre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [472]:
def return_most_common_venues(row, num_top_venues):
    
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
    

In [494]:
for ind in np.arange(toronto_grouped.shape[0]):
    venue_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)
    
venue_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Pizza Place,Restaurant,Skating Rink,Japanese Restaurant,Discount Store,Sushi Restaurant
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Bank,Bar,Italian Restaurant,Beer Store,Donut Shop,Sandwich Place,Park
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Park,Coffee Shop,Ski Area,Ice Cream Shop,Supermarket,Dog Run,Sushi Restaurant,Restaurant,Chinese Restaurant
3,Bayview Village,Park,Chinese Restaurant,Trail,Convenience Store,Pharmacy,Bank,Japanese Restaurant,Café,Coffee Shop,Donut Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Sushi Restaurant,Thai Restaurant,Bank,Indian Restaurant,Bridal Shop,Butcher,Intersection
5,Berczy Park,Coffee Shop,Café,Beer Bar,Creperie,Restaurant,Cocktail Bar,Japanese Restaurant,Park,Art Gallery,Sporting Goods Shop
6,"Birch Cliff, Cliffside West",Park,Convenience Store,Gym Pool,Farm,Bus Stop,Skating Rink,Café,General Entertainment,Diner,Restaurant
7,"Brockton, Parkdale Village, Exhibition Place",Café,Restaurant,Bakery,Gift Shop,Arts & Crafts Store,Indian Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Coffee Shop,French Restaurant
8,Business reply mail Processing Centre,Coffee Shop,Café,American Restaurant,Restaurant,Concert Hall,Theater,Hotel,Movie Theater,Plaza,Taco Place
9,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Gym,Italian Restaurant,Yoga Studio,Park,Spa,Brewery,Bakery,Restaurant,Beer Bar


# Run K means clustering

In [492]:
from sklearn.cluster import KMeans

In [495]:
kclusters = 5

toronto_group_clustering = toronto_grouped.drop('Neighborhood',1)

kmeans = KMeans(n_clusters = kclusters, random_state=42).fit(toronto_group_clustering)

kmeans.labels_[0:10]

array([4, 4, 4, 2, 1, 1, 2, 0, 1, 1], dtype=int32)

In [497]:
venue_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [533]:
venue_sorted.head(3)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Sandwich Place,Pizza Place,Restaurant,Skating Rink,Japanese Restaurant,Discount Store,Sushi Restaurant
1,4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Bank,Bar,Italian Restaurant,Beer Store,Donut Shop,Sandwich Place,Park
2,4,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Park,Coffee Shop,Ski Area,Ice Cream Shop,Supermarket,Dog Run,Sushi Restaurant,Restaurant,Chinese Restaurant


In [512]:
df_mapped_full = df_mapped.merge(how = 'left', right = venue_sorted.set_index('Neighborhood'), on='Neighborhood',indicator = True)

In [542]:
# drop where no clusters

df_mapped_full = df_mapped_full[df_mapped_full['Cluster Labels'].notna()]
df_mapped_full['Cluster Labels'] = df_mapped_full['Cluster Labels'].astype('int')

In [520]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [547]:
## map visualization

map_clusters = folium.Map(
    location=[toronto_data.lat,toronto_data.lng],zoom_start=10)


#set color scheme

x = np.arange(kclusters)

ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []


for lat,lng, poi, cluster in zip(df_mapped_full['Latitude'], df_mapped_full['Longitude'],
                                 df_mapped_full['Neighborhood'], df_mapped_full['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster:' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup=label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters
    

In [548]:
df_mapped_full

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
0,M3A,North York,Parkwoods,43.752935,-79.335641,2,Park,Bus Stop,Golf Course,Bank,Road,Train Station,Coffee Shop,Food & Drink Shop,Dentist's Office,Creperie,both
1,M4A,North York,Victoria Village,43.728102,-79.311890,4,Middle Eastern Restaurant,Pizza Place,Coffee Shop,Thai Restaurant,Grocery Store,Spa,Sporting Goods Shop,Bus Line,Sandwich Place,Chinese Restaurant,both
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1,Coffee Shop,Bakery,Park,Café,Athletics & Sports,Gym / Fitness Center,Pub,Restaurant,Spa,Gastropub,both
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,1,Clothing Store,Cosmetics Shop,Greek Restaurant,Furniture / Home Store,Restaurant,Toy / Game Store,Dessert Shop,Men's Store,American Restaurant,Sushi Restaurant,both
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1,Coffee Shop,Gastropub,Park,Sushi Restaurant,Bubble Tea Shop,Japanese Restaurant,Jewelry Store,Museum,Sandwich Place,Distribution Center,both
5,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953,2,Pharmacy,Playground,Park,Bank,Bakery,Japanese Restaurant,Café,Shopping Mall,Skating Rink,Grocery Store,both
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913,4,Fast Food Restaurant,Zoo Exhibit,Coffee Shop,Garden,Gas Station,Trail,Spa,Pizza Place,Hobby Shop,Cupcake Shop,both
7,M3B,North York,Don Mills,43.748900,-79.357220,4,Japanese Restaurant,Coffee Shop,Asian Restaurant,Bank,Italian Restaurant,Pizza Place,Fried Chicken Joint,Burger Joint,Restaurant,Chinese Restaurant,both
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529,4,Brewery,Fast Food Restaurant,Pizza Place,Intersection,Pharmacy,Coffee Shop,Office,Rock Climbing Spot,Café,Gastropub,both
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,1,Coffee Shop,Gastropub,Theater,Restaurant,Plaza,Diner,Japanese Restaurant,Bubble Tea Shop,Tea Room,Seafood Restaurant,both


## Examine clusters

In [549]:
df_mapped_full.loc[df_mapped_full['Cluster Labels'] == 0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
25,M6G,Downtown Toronto,Christie,43.668602,-79.420387,0,Café,Korean Restaurant,Coffee Shop,Grocery Store,Indian Restaurant,Cocktail Bar,Restaurant,Ethiopian Restaurant,Taco Place,Latin American Restaurant,both
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.664910,-79.438664,0,Café,Coffee Shop,Bar,Bakery,Brewery,Cocktail Bar,Italian Restaurant,Portuguese Restaurant,Toy / Game Store,Gastropub,both
43,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.639922,-79.431240,0,Café,Restaurant,Bakery,Gift Shop,Arts & Crafts Store,Indian Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Coffee Shop,French Restaurant,both
57,M9M,North York,"Humberlea, Emery",43.733682,-79.544381,0,Coffee Shop,Latin American Restaurant,Nightclub,Golf Course,Discount Store,Grocery Store,Café,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,both
69,M6P,West Toronto,"High Park, The Junction South",43.658720,-79.463150,0,Café,Bar,Thai Restaurant,Sushi Restaurant,Italian Restaurant,Grocery Store,Gastropub,Flea Market,Breakfast Spot,Coffee Shop,both
81,M6S,West Toronto,"Runnymede, Swansea",43.649725,-79.482692,0,Café,Coffee Shop,Italian Restaurant,Bakery,Falafel Restaurant,Bank,Sushi Restaurant,Park,Bookstore,Pub,both


In [550]:
df_mapped_full.loc[df_mapped_full['Cluster Labels'] == 1]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,1,Coffee Shop,Bakery,Park,Café,Athletics & Sports,Gym / Fitness Center,Pub,Restaurant,Spa,Gastropub,both
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,1,Clothing Store,Cosmetics Shop,Greek Restaurant,Furniture / Home Store,Restaurant,Toy / Game Store,Dessert Shop,Men's Store,American Restaurant,Sushi Restaurant,both
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,1,Coffee Shop,Gastropub,Park,Sushi Restaurant,Bubble Tea Shop,Japanese Restaurant,Jewelry Store,Museum,Sandwich Place,Distribution Center,both
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,1,Coffee Shop,Gastropub,Theater,Restaurant,Plaza,Diner,Japanese Restaurant,Bubble Tea Shop,Tea Room,Seafood Restaurant,both
15,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,1,Café,Gastropub,Coffee Shop,Seafood Restaurant,Cocktail Bar,Theater,Cosmetics Shop,Italian Restaurant,Gym,New American Restaurant,both
19,M4E,East Toronto,The Beaches,43.678148,-79.295349,1,Pub,Coffee Shop,Pizza Place,Bar,Caribbean Restaurant,Breakfast Spot,Japanese Restaurant,Tea Room,Sandwich Place,Indie Movie Theater,both
20,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,1,Coffee Shop,Café,Beer Bar,Creperie,Restaurant,Cocktail Bar,Japanese Restaurant,Park,Art Gallery,Sporting Goods Shop,both
24,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653,1,Coffee Shop,Café,Theater,Plaza,Japanese Restaurant,Smoke Shop,Middle Eastern Restaurant,Ice Cream Shop,Gastropub,Modern European Restaurant,both
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650542,-79.384116,1,Café,Coffee Shop,Theater,Concert Hall,American Restaurant,Gym,Restaurant,Shopping Mall,Furniture / Home Store,Steakhouse,both
35,M4J,East York,East Toronto,43.687046,-79.333890,1,Coffee Shop,Café,Ethiopian Restaurant,Sandwich Place,Breakfast Spot,Beer Bar,Gastropub,Pizza Place,American Restaurant,Dim Sum Restaurant,both


In [551]:
df_mapped_full.loc[df_mapped_full['Cluster Labels'] == 2]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
0,M3A,North York,Parkwoods,43.752935,-79.335641,2,Park,Bus Stop,Golf Course,Bank,Road,Train Station,Coffee Shop,Food & Drink Shop,Dentist's Office,Creperie,both
5,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953,2,Pharmacy,Playground,Park,Bank,Bakery,Japanese Restaurant,Café,Shopping Mall,Skating Rink,Grocery Store,both
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650023,-79.554089,2,Pizza Place,Park,Convenience Store,Mexican Restaurant,Hotel,Fish & Chips Shop,Clothing Store,Bank,Gym,Grocery Store,both
21,M6E,York,Caledonia-Fairbanks,43.689118,-79.450650,2,Pizza Place,Pharmacy,Park,Grocery Store,Cosmetics Shop,Coffee Shop,Discount Store,Mexican Restaurant,Bus Line,Beer Store,both
22,M1G,Scarborough,Woburn,43.771545,-79.218135,2,Park,Indian Restaurant,Coffee Shop,Pharmacy,Zoo Exhibit,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Fabric Shop,both
27,M2H,North York,Hillcrest Village,43.802556,-79.356566,2,Pharmacy,Park,Sandwich Place,Coffee Shop,Chinese Restaurant,Residential Building (Apartment / Condo),Bank,Bakery,Grocery Store,Dog Run,both
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.623750,-79.369200,2,Harbor / Marina,Park,Theme Park,Pizza Place,BBQ Joint,Farm,Music Venue,Beach,Scenic Lookout,Burger Joint,both
39,M2K,North York,Bayview Village,43.780607,-79.376921,2,Park,Chinese Restaurant,Trail,Convenience Store,Pharmacy,Bank,Japanese Restaurant,Café,Coffee Shop,Donut Shop,both
45,M2L,North York,"York Mills, Silver Hills",43.749360,-79.380690,2,Bank,Park,Cosmetics Shop,Bus Stop,Sandwich Place,Baseball Field,Supermarket,Coffee Shop,Dentist's Office,Butcher,both
58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696448,-79.265642,2,Park,Convenience Store,Gym Pool,Farm,Bus Stop,Skating Rink,Café,General Entertainment,Diner,Restaurant,both


In [552]:
df_mapped_full.loc[df_mapped_full['Cluster Labels'] == 3]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,3,Breakfast Spot,Park,Burger Joint,Bar,Cupcake Shop,Curling Ice,Electronics Store,Ethiopian Restaurant,Fabric Shop,Falafel Restaurant,both
